#Analisando candles do dolar de 5 minutos
## usando RNN e MSE

In [0]:
!pip install tensorflow==1.6.0
!pip show tensorflow  # 1.6.0
!pip show keras  # 2.1.5

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler

In [0]:
!wget https://raw.githubusercontent.com/juanengml/Previs-es-do-mercado-de-a-es-com-LSTM-em-Python/master/CANDLE%20DOLAR%205%20MIN.csv

In [0]:
!ls 

In [0]:
df = pd.read_csv("CANDLE DOLAR 5 MIN.csv")

In [0]:
df = df.sort_values('dt_registro')

In [0]:
df.head()

## Nesse processo vamos usar as colunas low e high

In [0]:
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),(df['low']+df['high'])/2.0)
plt.xticks(range(0,df.shape[0],500),df['dt_registro'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.show()

##Definindo preços altos, baixos e media 

In [0]:
preco_alto = df.loc[:,'high'].as_matrix()
preco_baixo = df.loc[:,'low'].as_matrix()
media_precos = (preco_alto+preco_baixo)/2.0

In [0]:
len(media_precos)/2

## Splinting dados de treino e teste

In [0]:
data_treino = media_precos[:11000]
data_teste = media_precos[11000:]

## O que é MinMaxScaller ? 
Para cada valor em um recurso, o MinMaxScaler subtrai o valor mínimo no recurso e depois divide pelo intervalo. O intervalo é a diferença entre o máximo original e o mínimo original.

MinMaxScaler preserva a forma da distribuição original. Não altera significativamente as informações incorporadas nos dados originais. Observe que o MinMaxScaler não reduz a importância dos valores discrepantes.O intervalo padrão para o recurso retornado pelo MinMaxScaler é de 0 a 1.

In [0]:
scaler = MinMaxScaler()
data_treino = data_treino.reshape(-1,1)
data_teste = data_teste.reshape(-1,1)

#### Normalizando o último bit de dados restante

In [0]:
tamano_da_janela_ = 2500
for di in range(0,10000,tamano_da_janela_):
    scaler.fit(data_treino[di:di+tamano_da_janela_,:])
    data_treino[di:di+tamano_da_janela_,:] = scaler.transform(data_treino[di:di+tamano_da_janela_,:])

scaler.fit(data_treino[di+tamano_da_janela_:,:])
data_treino[di+tamano_da_janela_:,:] = scaler.transform(data_treino[di+tamano_da_janela_:,:])


### Normalizar dados de teste



In [0]:
data_treino = data_treino.reshape(-1)

#  Normalizar dados de teste
data_teste = scaler.transform(data_teste).reshape(-1)

In [0]:
EMA = 0.0
gamma = 0.1
for ti in range(11000):
  EMA = gamma*data_treino[ti] + (1-gamma)*EMA
  data_treino[ti] = EMA

all_mid_data = np.concatenate([data_treino,data_teste],axis=0)

## Plot 

In [0]:
janela = 100
N = data_treino.size
std_avg_predictions = []
std_avg_x = []
mse_errors = []
for pred_idx in range(janela,N):
    if pred_idx >= N:
        date = dt.datetime.strptime(k, '%Y-%m-%d').date() + dt.timedelta(days=1)
    else:
        date = df.loc[pred_idx,'dt_registro']
    std_avg_predictions.append(np.mean(data_treino[pred_idx-janela:pred_idx]))
    mse_errors.append((std_avg_predictions[-1]-data_treino[pred_idx])**2)
    std_avg_x.append(date)

In [0]:
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),all_mid_data,color='b',label='True')
plt.plot(range(janela,N),std_avg_predictions,color='orange',label='Prediction')
#plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()

In [0]:
window_size = 100
N = data_treino.size
run_avg_predictions = []
run_avg_x = []
mse_errors = []
running_mean = 0.0
run_avg_predictions.append(running_mean)
decay = 0.5

## Tafuq MSE ? 
O erro médio quadrático (MSE) de um estimador mede a média dos quadrados dos erros,isto é, a diferença quadrática média entre os valores estimados e o valor real. É uma função de risco, correspondente ao valor esperado da perda de erro ao quadrado. É sempre não negativo e valores próximos de zero são melhores. O MSE é o segundo momento do erro (sobre a origem) e, portanto, incorpora a variação do estimador e seu viés.



In [0]:
for pred_idx in range(1,N):
    running_mean = running_mean*decay + (1.0-decay)*data_treino[pred_idx-1]
    run_avg_predictions.append(running_mean)
    mse_errors.append((run_avg_predictions[-1]-data_treino[pred_idx])**2)
    run_avg_x.append(date)

In [0]:
print('MSE error - EMA: %.5f'%(0.5*np.mean(mse_errors)))

In [0]:
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),all_mid_data,color='b',label='True')
plt.plot(range(0,N),run_avg_predictions,color='orange', label='Prediction')
#plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()


# Usando LSTM para Stock Prediciton

In [0]:
class DataGeneratorSeq(object):

    def __init__(self,prices,batch_size,num_unroll):
        self._prices = prices
        self._prices_length = len(self._prices) - num_unroll
        self._batch_size = batch_size
        self._num_unroll = num_unroll
        self._segments = self._prices_length //self._batch_size
        self._cursor = [offset * self._segments for offset in range(self._batch_size)]

    def next_batch(self):

        batch_data = np.zeros((self._batch_size),dtype=np.float32)
        batch_labels = np.zeros((self._batch_size),dtype=np.float32)

        for b in range(self._batch_size):
            if self._cursor[b]+1>=self._prices_length:
                #self._cursor[b] = b * self._segments
                self._cursor[b] = np.random.randint(0,(b+1)*self._segments)

            batch_data[b] = self._prices[self._cursor[b]]
            batch_labels[b]= self._prices[self._cursor[b]+np.random.randint(0,5)]

            self._cursor[b] = (self._cursor[b]+1)%self._prices_length

        return batch_data,batch_labels

    def unroll_batches(self):

        unroll_data,unroll_labels = [],[]
        init_data, init_label = None,None
        for ui in range(self._num_unroll):

            data, labels = self.next_batch()    

            unroll_data.append(data)
            unroll_labels.append(labels)

        return unroll_data, unroll_labels

    def reset_indices(self):
        for b in range(self._batch_size):
            self._cursor[b] = np.random.randint(0,min((b+1)*self._segments,self._prices_length-1))


In [0]:

dg = DataGeneratorSeq(data_treino,5,5)
u_data, u_labels = dg.unroll_batches()

for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):   
    print('\n\nUnrolled index %d'%ui)
    dat_ind = dat
    lbl_ind = lbl
    print('\tInputs: ',dat )
    print('\n\tOutput:',lbl)    

In [0]:
D = 1 # Dimensionalidade dos dados. Como seus dados são 1-D, isso seria 1
num_unrollings = 50 # NuEm várias etapas do tempo, você olha para o futuro.
batch_size = 500 # Número de amostras em um lote
num_nodes = [200,200,150] # Número de nós ocultos em cada camada da pilha profunda do LSTM que estamos usando
n_layers = len(num_nodes) # Numero de camadas
dropout = 0.2 # dropout

tf.reset_default_graph() # Isso é importante caso você execute isso várias vezes

In [0]:
# entrada de dados
train_inputs, train_outputs = [],[]

# Você desenrola a entrada ao longo do tempo, definindo espaços reservados para cada etapa
for ui in range(num_unrollings):
    train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,D],name='train_inputs_%d'%ui))
    train_outputs.append(tf.placeholder(tf.float32, shape=[batch_size,1], name = 'train_outputs_%d'%ui))


In [0]:
lstm_cells = [
    tf.contrib.rnn.LSTMCell(num_units=num_nodes[li],
         state_is_tuple=True,initializer= tf.contrib.layers.xavier_initializer()
                           )
         for li in range(n_layers)]

In [0]:

drop_lstm_cells = [tf.contrib.rnn.DropoutWrapper(
    lstm, input_keep_prob=1.0,output_keep_prob=1.0-dropout, state_keep_prob=1.0-dropout
) for lstm in lstm_cells]
drop_multi_cell = tf.contrib.rnn.MultiRNNCell(drop_lstm_cells)
multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)

w = tf.get_variable('w',shape=[num_nodes[-1], 1], initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable('b',initializer=tf.random_uniform([1],-0.1,0.1))

In [0]:
# Crie variáveis ​​de estado da célula e estado oculto para manter o estado do LSTM

c, h = [],[]
initial_state = []
for li in range(n_layers):
  c.append(tf.Variable(tf.zeros([batch_size, num_nodes[li]]), trainable=False))
  h.append(tf.Variable(tf.zeros([batch_size, num_nodes[li]]), trainable=False))
  initial_state.append(tf.contrib.rnn.LSTMStateTuple(c[li], h[li]))

all_inputs = tf.concat([tf.expand_dims(t,0) for t in train_inputs],axis=0)
all_lstm_outputs, state = tf.nn.dynamic_rnn(
    drop_multi_cell, all_inputs, initial_state=tuple(initial_state),
    time_major = True, dtype=tf.float32)
all_lstm_outputs = tf.reshape(all_lstm_outputs, [batch_size*num_unrollings,num_nodes[-1]])
all_outputs = tf.nn.xw_plus_b(all_lstm_outputs,w,b)
split_outputs = tf.split(all_outputs,num_unrollings,axis=0)


In [0]:
# Ao calcular a perda, você precisa ter cuidado com a forma exata, porque calcula
# perda de todas as etapas desenroladas ao mesmo tempo
# Portanto, considere o erro médio ou cada lote e obtenha a soma disso em todas as etapas desenroladas

print('Perdade Treino')
loss = 0.0
with tf.control_dependencies([tf.assign(c[li], state[li][0]) for li in range(n_layers)]+
                             [tf.assign(h[li], state[li][1]) for li in range(n_layers)]):
  for ui in range(num_unrollings):
    loss += tf.reduce_mean(0.5*(split_outputs[ui]-train_outputs[ui])**2)

print('Rate de Aprendizado em opecarao decay')
global_step = tf.Variable(0, trainable=False)
inc_gstep = tf.assign(global_step,global_step + 1)
tf_learning_rate = tf.placeholder(shape=None,dtype=tf.float32)
tf_min_learning_rate = tf.placeholder(shape=None,dtype=tf.float32)

learning_rate = tf.maximum(
    tf.train.exponential_decay(tf_learning_rate, global_step, decay_steps=1, decay_rate=0.5, staircase=True),
    tf_min_learning_rate)

# Otimização.
print('TF Otimizacao')
optimizer = tf.train.AdamOptimizer(learning_rate)
gradients, v = zip(*optimizer.compute_gradients(loss))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
optimizer = optimizer.apply_gradients(
    zip(gradients, v))

print('\tDahora DOne')

In [0]:
print('Definindo funcoes TF relacionadas a previsao')

sample_inputs = tf.placeholder(tf.float32, shape=[1,D])

# Manutenção do estado LSTM para o estágio de previsão
sample_c, sample_h, initial_sample_state = [],[],[]
for li in range(n_layers):
  sample_c.append(tf.Variable(tf.zeros([1, num_nodes[li]]), trainable=False))
  sample_h.append(tf.Variable(tf.zeros([1, num_nodes[li]]), trainable=False))
  initial_sample_state.append(tf.contrib.rnn.LSTMStateTuple(sample_c[li],sample_h[li]))


In [0]:
reset_sample_states = tf.group(*[tf.assign(sample_c[li],tf.zeros([1, num_nodes[li]])) for li in range(n_layers)],
                               *[tf.assign(sample_h[li],tf.zeros([1, num_nodes[li]])) for li in range(n_layers)])

sample_outputs, sample_state = tf.nn.dynamic_rnn(multi_cell, tf.expand_dims(sample_inputs,0),
                                   initial_state=tuple(initial_sample_state),
                                   time_major = True,
                                   dtype=tf.float32)

with tf.control_dependencies([tf.assign(sample_c[li],sample_state[li][0]) for li in range(n_layers)]+
                              [tf.assign(sample_h[li],sample_state[li][1]) for li in range(n_layers)]):  
  sample_prediction = tf.nn.xw_plus_b(tf.reshape(sample_outputs,[1,-1]), w, b)

print('\tDone cacete')


In [0]:
from datetime import datetime

In [0]:
epochs = 20
valid_summary = 1 # Intervalo em que você faz previsões de teste
n_predict_once = 50 # Número de etapas que você prevê continuamente
train_seq_length = data_treino.size # Comprimento total dos dados de treinamento
train_mse_ot = [] # Acumular perdas de trem
test_mse_ot = [] # Acumular perda de teste
predictions_over_time = [] # Acumular previsões
session = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Usado para diminuir a taxa de aprendizado
loss_nondecrease_count = 0
loss_nondecrease_threshold = 2 # Se o erro de teste não aumentar nessas etapas, diminua a taxa de aprendizado
print('Inicio do role.....')
average_loss = 0
# Definir gerador de dados

data_gen = DataGeneratorSeq(data_treino,batch_size,num_unrollings)
x_axis_seq = []
# Pontos pelos quais você inicia suas previsões de teste
test_points_seq = np.arange(11000,12000,50).tolist()
for ep in range(epochs):       
    # treino
    print(datetime.now())
    for step in range(train_seq_length//batch_size):
        u_data, u_labels = data_gen.unroll_batches()
        feed_dict = {}
        for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):            
            feed_dict[train_inputs[ui]] = dat.reshape(-1,1)
            feed_dict[train_outputs[ui]] = lbl.reshape(-1,1)
        feed_dict.update({tf_learning_rate: 0.0001, tf_min_learning_rate:0.000001})
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
    # Validação
    if (ep+1) % valid_summary == 0:
      average_loss = average_loss/(valid_summary*(train_seq_length//batch_size))
      if (ep+1)%valid_summary==0:
        print('Media de perda step %d: %f' % (ep+1, average_loss))
      train_mse_ot.append(average_loss)
      average_loss = 0 
      predictions_seq = []
      mse_test_loss_seq = []
      # Atualizacao de Stado e Predictions
      for w_i in test_points_seq:
        mse_test_loss = 0.0
        our_predictions = []

        if (ep+1)-valid_summary==0:
          x_axis=[]
        for tr_i in range(w_i-num_unrollings+1,w_i-1):
          current_price = all_mid_data[tr_i]
          feed_dict[sample_inputs] = np.array(current_price).reshape(1,1)    
          _ = session.run(sample_prediction,feed_dict=feed_dict)

        feed_dict = {}
        current_price = all_mid_data[w_i-1]
        feed_dict[sample_inputs] = np.array(current_price).reshape(1,1)
        for pred_i in range(n_predict_once):
          pred = session.run(sample_prediction,feed_dict=feed_dict)
          our_predictions.append(np.asscalar(pred))
          feed_dict[sample_inputs] = np.asarray(pred).reshape(-1,1)
          if (ep+1)-valid_summary==0:
            x_axis.append(w_i+pred_i)
          mse_test_loss += 0.5*(pred-all_mid_data[w_i+pred_i])**2
          
        session.run(reset_sample_states)
        predictions_seq.append(np.array(our_predictions))
        mse_test_loss /= n_predict_once
        mse_test_loss_seq.append(mse_test_loss)
        if (ep+1)-valid_summary==0:
          x_axis_seq.append(x_axis)
      current_test_mse = np.mean(mse_test_loss_seq)
      if len(test_mse_ot)>0 and current_test_mse > min(test_mse_ot):
          loss_nondecrease_count += 1
      else:
          loss_nondecrease_count = 0
      if loss_nondecrease_count > loss_nondecrease_threshold :
            session.run(inc_gstep)
            loss_nondecrease_count = 0
            print('\tlearning rate by 0.5')
      test_mse_ot.append(current_test_mse)
      print('\tTeste MSE: %.5f'%np.mean(mse_test_loss_seq))
      predictions_over_time.append(predictions_seq)
      print('\tFim cacete de Predictions',datetime.now())


# Visualização da Precisão

In [0]:
best_prediction_epoch = 20 # substitua isso pela época em que você obteve os melhores resultados ao executar o código de plotagem

plt.figure(figsize = (18,18))
plt.subplot(2,1,1)
plt.plot(range(df.shape[0]),all_mid_data,color='b')

# Traçar como as previsões mudam ao longo do tempo
# Plotar previsões mais antigas com alfa baixo e previsões mais recentes com alfa alto

start_alpha = 0.25
alpha  = np.arange(start_alpha,1.1,(1.0-start_alpha)/len(predictions_over_time[::3]))
for p_i,p in enumerate(predictions_over_time[::3]):
    for xval,yval in zip(x_axis_seq,p):
        plt.plot(xval,yval,color='r',alpha=alpha[p_i])

plt.title('Evolution of Test Predictions Over Time',fontsize=18)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.xlim(11000,12500)

plt.subplot(2,1,2)

# Como prever a melhor previsão de teste que você obteve

plt.plot(range(df.shape[0]),all_mid_data,color='b')
for xval,yval in zip(x_axis_seq,predictions_over_time[best_prediction_epoch]):
    plt.plot(xval,yval,color='r')

plt.title('Best Test Predictions Over Time',fontsize=18)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.xlim(11000,12500)
plt.show()

---

# FONTE DE PESQUISA

MinMaxScale - https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02

MSE - https://www.geeksforgeeks.org/python-mean-squared-error/

LSTM - http://deeplearningbook.com.br/arquitetura-de-redes-neurais-long-short-term-memory/